In [160]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmcrameri import cm

import os
from pathlib import Path
import shutil
import gzip
import re
import csv

import timeit

In [161]:
data_path = 'data/updated_simulation_data'
output_file_all_all = 'processed_data/collisions_all_data.csv'
output_file_all = 'processed_data/allcollisions_GiantBH_data.csv'
output_file_nocollision = 'processed_data/nocollision_GiantBH_data.csv'
output_file_iscollision = 'processed_data/iscollision_GiantBH_data.csv'
#header = "#N,#rv,#rg,#z,#t_snapshot[myr],#M1[MSUN],#M2[MSUN],#k1,#k2,#id1,#id2,#sma[AU],#ecc,#bin_star_radius0[RSUN],#bin_star_radius1[RSUN],#snapshot, #roche_lobe1_calc[RSUN], #roche_lobe2_calc[RSUN],#radrol0,#radrol1"

output_file_initial = 'processed_data/initial_GiantBH_data.csv'
output_file_initial_all = 'processed_data/initial_all_GiantBH_data.csv'


In [162]:
def get_folder_names(directory):
    try:
        # List all entries in the specified directory
        entries = os.listdir(directory)
        
        # Filter out only the folders
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory, entry))]
        
        return folders
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

# Specify the directory
directory_path = r'data/updated_simulation_data'

# Get the list of folder names
str_numbers = get_folder_names(directory_path)

# Print the list of folder names
print("Folder names:", str_numbers)
print("Number of folders:", len(str_numbers))

Folder names: ['N16_rv0.5_rg2.0_z0.002', 'N16_rv0.5_rg2.0_z0.02', 'N16_rv0.5_rg20.0_z0.002', 'N16_rv0.5_rg20.0_z0.02', 'N16_rv0.5_rg8.0_z0.002', 'N16_rv0.5_rg8.0_z0.02', 'N16_rv1.0_rg2.0_z0.0002', 'N16_rv1.0_rg2.0_z0.002', 'N16_rv1.0_rg2.0_z0.02', 'N16_rv1.0_rg20.0_z0.0002', 'N16_rv1.0_rg20.0_z0.002', 'N16_rv1.0_rg20.0_z0.02', 'N16_rv1.0_rg8.0_z0.0002', 'N16_rv1.0_rg8.0_z0.002', 'N16_rv1.0_rg8.0_z0.02', 'N16_rv2.0_rg2.0_z0.0002', 'N16_rv2.0_rg2.0_z0.002', 'N16_rv2.0_rg2.0_z0.02', 'N16_rv2.0_rg20.0_z0.0002', 'N16_rv2.0_rg20.0_z0.002', 'N16_rv2.0_rg20.0_z0.02', 'N16_rv2.0_rg8.0_z0.0002', 'N16_rv2.0_rg8.0_z0.002', 'N16_rv2.0_rg8.0_z0.02', 'N16_rv4.0_rg2.0_z0.0002', 'N16_rv4.0_rg2.0_z0.002', 'N16_rv4.0_rg2.0_z0.02', 'N16_rv4.0_rg20.0_z0.0002', 'N16_rv4.0_rg20.0_z0.002', 'N16_rv4.0_rg20.0_z0.02', 'N16_rv4.0_rg8.0_z0.0002', 'N16_rv4.0_rg8.0_z0.002', 'N16_rv4.0_rg8.0_z0.02', 'N2.0_rv0.5_rg2.0_z0.0002', 'N2.0_rv0.5_rg2.0_z0.002', 'N2.0_rv0.5_rg2.0_z0.02', 'N2.0_rv0.5_rg20.0_z0.0002', 'N2.0_rv0

In [163]:
def get_rows_of_interest(input_file, ids_of_interest, collision_times):
    # Set of star IDs of interest for quick lookup
    ids_set = set(map(float, ids_of_interest))
    rows_of_interest = []

    with open(input_file, 'r') as infile:
        for line in infile:
            # Skip comments and empty lines
            if line.startswith('#') or not line.strip():
                continue

            # Split the line into columns
            columns = line.split()

            # Check if the row is a binary system or a single star
            id_num = float(columns[0])
            binflag = float(columns[7])

            if id_num in ids_set or (binflag == 1 and (float(columns[10]) in ids_set or float(columns[11]) in ids_set)):
                # Determine the collision time to append
                collision_time = collision_times.get(id_num, 'N/A')
                if binflag == 1:
                    partner_id1 = float(columns[10])
                    partner_id2 = float(columns[11])
                    partner_collision_time = collision_times.get(partner_id1, collision_times.get(partner_id2, 'N/A'))
                    if collision_time == 'N/A':
                        collision_time = partner_collision_time

                    # If both IDs are in the list of interest, duplicate the row and modify
                    if partner_id1 in ids_set and partner_id2 in ids_set:
                        row1 = columns.copy()
                        row2 = columns.copy()

                        row1[10] = '-100'
                        row1[11] = str(partner_id2)

                        row2[10] = str(partner_id1)
                        row2[11] = '-100'

                        # Swap columns for row1 if necessary
                        if float(row1[11]) in ids_set:
                            temp = row1[10]
                            row1[10] = row1[11]
                            row1[11] = temp

                            temp = row1[8]
                            row1[8] = row1[9]
                            row1[9] = temp

                            temp = row1[17]
                            row1[17] = row1[18]
                            row1[18] = temp

                        # Update columns[0] and columns[1]
                        row1[0] = row1[10]
                        row1[1] = row1[8]

                        # Copy columns[17] to columns[14] if id0 is not '-100'
                        if row1[10] != '-100':
                            row1[14] = row1[17]

                        # Swap columns for row2 if necessary
                        if float(row2[11]) in ids_set:
                            temp = row2[10]
                            row2[10] = row2[11]
                            row2[11] = temp

                            temp = row2[8]
                            row2[8] = row2[9]
                            row2[9] = temp

                            temp = row2[17]
                            row2[17] = row2[18]
                            row2[18] = temp

                        # Update columns[0] and columns[1]
                        row2[0] = row2[10]
                        row2[1] = row2[8]

                        # Copy columns[17] to columns[14] if id0 is not '-100'
                        if row2[10] != '-100':
                            row2[14] = row2[17]

                        rows_of_interest.append(row1 + [collision_time])
                        rows_of_interest.append(row2 + [collision_time])

                    else:
                        # Replace the non-interest ID with '-100'
                        if partner_id1 in ids_set and partner_id2 not in ids_set:
                            columns[11] = '-100'
                        elif partner_id2 in ids_set and partner_id1 not in ids_set:
                            columns[10] = '-100'

                        # Swap columns if the id of interest is in columns[11]
                        if float(columns[11]) in ids_set:
                            temp = columns[10]
                            columns[10] = columns[11]
                            columns[11] = temp

                            temp = columns[8]
                            columns[8] = columns[9]
                            columns[9] = temp

                            temp = columns[17]
                            columns[17] = columns[18]
                            columns[18] = temp

                        # Update columns[0] and columns[1]
                        columns[0] = columns[10]
                        columns[1] = columns[8]

                        # Copy columns[17] to columns[14] if id0 is not '-100'
                        if columns[10] != '-100':
                            columns[14] = columns[17]

                        rows_of_interest.append(columns + [collision_time])
                else:
                    rows_of_interest.append(columns + [collision_time])

    return rows_of_interest

In [164]:
id_Giant = [2, 3, 4, 5, 6, 7, 8, 9]
id_BH = [14]

all_data = []

for str_number in str_numbers:
    # Go through each model individually
    initial_path = f'data/updated_simulation_data/{str_number}/info/initial.snap0000.dat'
    conv_path = f'data/updated_simulation_data/{str_number}/info/initial.conv.sh'

    # Read the file
    with open(conv_path, 'r') as file:
        lines = file.readlines()

    # Extract the value for timeunitsmyr
    for line in lines:
        if line.startswith('timeunitsmyr'):
            _, value = line.split('=')
            timeunitsmyr = float(value.strip())
            break

    if os.path.exists(initial_path):
        # Grab the values of the parameters from the file path
        pattern = r"N(?P<N>[\d.]+)_rv(?P<rv>[\d.]+)_rg(?P<rg>[\d.]+)_z(?P<z>[\d.]+)"
        match = re.search(pattern, str_number)
        if match:
            mod_params = [float(match.group('N')), float(match.group('rv')), float(match.group('rg')), float(match.group('z'))]
        else:
            print(f"Pattern {str_number} not found in the file path.")
            continue

        # List to hold the values from the 6th column and a dictionary for collision times
        id_Giant = []
        collision_times = {}

        # Open the CSV file and read its contents
        with open(output_file_nocollision, mode='r') as file:
            reader = csv.reader(file)
            
            # Skip the header row
            header = next(reader)
            
            # Iterate through each row in the CSV file
            for row in reader:
                # Check if the first four columns match the model parameters
                if (float(row[0]) == float(mod_params[0]) and float(row[1]) == float(mod_params[1]) and 
                    float(row[2]) == float(mod_params[2]) and float(row[3]) == float(mod_params[3])):
                    # Append the value from the 6th column to the list
                    id_Giant.append(row[10])
                    # Store the collision time
                    collision_times[float(row[10])] = row[4]
        
        # Get rows of interest with appended collision times
        result = get_rows_of_interest(initial_path, id_Giant, collision_times)

        mod_params.append('0')

        # Append the model parameters to the start of each sublist
        data_list = [mod_params + sublist if type(sublist) is list else mod_params + [sublist] for sublist in result]

        # Append the data to the list of all data
        if len(data_list) > 0:
            all_data += data_list
        
        print(f"Finished {str_number}. All data: {len(all_data)}. Data list: {len(data_list)}")
    else:
        print(f"No such file {initial_path}")
        


Finished N16_rv0.5_rg2.0_z0.002. All data: 143. Data list: 143
Finished N16_rv0.5_rg2.0_z0.02. All data: 625. Data list: 482
Finished N16_rv0.5_rg20.0_z0.002. All data: 771. Data list: 146
Finished N16_rv0.5_rg20.0_z0.02. All data: 1318. Data list: 547
Finished N16_rv0.5_rg8.0_z0.002. All data: 1433. Data list: 115
Finished N16_rv0.5_rg8.0_z0.02. All data: 1951. Data list: 518
Finished N16_rv1.0_rg2.0_z0.0002. All data: 1971. Data list: 20
Finished N16_rv1.0_rg2.0_z0.002. All data: 2002. Data list: 31
Finished N16_rv1.0_rg2.0_z0.02. All data: 2112. Data list: 110
Finished N16_rv1.0_rg20.0_z0.0002. All data: 2130. Data list: 18
Finished N16_rv1.0_rg20.0_z0.002. All data: 2167. Data list: 37
Finished N16_rv1.0_rg20.0_z0.02. All data: 2267. Data list: 100
Finished N16_rv1.0_rg8.0_z0.0002. All data: 2288. Data list: 21
Finished N16_rv1.0_rg8.0_z0.002. All data: 2326. Data list: 38
Finished N16_rv1.0_rg8.0_z0.02. All data: 2438. Data list: 112
Finished N16_rv2.0_rg2.0_z0.0002. All data: 243

In [165]:
headers = ["N", "rv", "rg", "z", "t_snapshot[myr]", "id", "m[MSUN]", "r", "vr", "vt", "E", "J", "binflag", "m0[MSUN]", "m1[MSUN]", "id0", "id1", "a[AU]", "e", "startype", "luminosity[LSUN]", "radius[RSUN]", "bin_startype0", "bin_startype1", "bin_star_lum0[LSUN]", "bin_star_lum1[LSUN]", "bin_star_radius0[RSUN]", "bin_star_radius1[RSUN]", "bin.Eb", "eta", "star.phi", "rad0", "rad1", "tb", "lum0", "lum1", "massc0", "massc1", "radc0", "radc1", "menv0", "menv1", "renv0", "renv1", "tms0", "tms1", "dmdt0", "dmdt1", "radrol0", "radrol1", "ospin0", "ospin1", "B0", "B1", "formation0", "formation1", "bacc0", "bacc1", "tacc0", "tacc1", "mass0_0", "mass0_1", "epoch0", "epoch1", "ospin", "B", "formation", "tcoll"]


with open(output_file_initial, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    writer.writerows(all_data)

In [166]:
id_Giant = [2, 3, 4, 5, 6, 7, 8, 9]
id_BH = [14]

all_data_all = []


for str_number in str_numbers:
    # Go through each model individually
    initial_path = f'data/updated_simulation_data/{str_number}/info/initial.snap0000.dat'
    conv_path = f'data/updated_simulation_data/{str_number}/info/initial.conv.sh'

    # Read the file
    with open(conv_path, 'r') as file:
        lines = file.readlines()

    # Extract the value for timeunitsmyr
    for line in lines:
        if line.startswith('timeunitsmyr'):
            _, value = line.split('=')
            timeunitsmyr = float(value.strip())
            break

    if os.path.exists(initial_path):
        # Grab the values of the parameters from the file path
        pattern = r"N(?P<N>[\d.]+)_rv(?P<rv>[\d.]+)_rg(?P<rg>[\d.]+)_z(?P<z>[\d.]+)"
        match = re.search(pattern, str_number)
        if match:
            mod_params = [float(match.group('N')), float(match.group('rv')), float(match.group('rg')), float(match.group('z'))]
        else:
            print(f"Pattern {str_number} not found in the file path.")
            continue

        # List to hold the values from the 6th column and a dictionary for collision times
        id_Giant = []
        collision_times = {}

        # Open the CSV file and read its contents
        with open(output_file_all, mode='r') as file:
            reader = csv.reader(file)
            
            # Skip the header row
            header = next(reader)
            
            # Iterate through each row in the CSV file
            for row in reader:
                # Check if the first four columns match the model parameters
                if (float(row[0]) == float(mod_params[0]) and float(row[1]) == float(mod_params[1]) and 
                    float(row[2]) == float(mod_params[2]) and float(row[3]) == float(mod_params[3])):
                    # Append the value from the 6th column to the list
                    id_Giant.append(row[10])
                    # Store the collision time
                    collision_times[float(row[10])] = row[4]
        
        # Get rows of interest with appended collision times
        result = get_rows_of_interest(initial_path, id_Giant, collision_times)

        mod_params.append('0')

        # Append the model parameters to the start of each sublist
        data_list = [mod_params + sublist if type(sublist) is list else mod_params + [sublist] for sublist in result]

        # Append the data to the list of all data
        if len(data_list) > 0:
            all_data += data_list
        
        print(f"Finished {str_number}. All data: {len(all_data)}. Data list: {len(data_list)}")
        

    else:
        print(f"No such file {initial_path}")
        

Finished N16_rv0.5_rg2.0_z0.002. All data: 4909. Data list: 159
Finished N16_rv0.5_rg2.0_z0.02. All data: 5498. Data list: 589
Finished N16_rv0.5_rg20.0_z0.002. All data: 5653. Data list: 155
Finished N16_rv0.5_rg20.0_z0.02. All data: 6302. Data list: 649
Finished N16_rv0.5_rg8.0_z0.002. All data: 6432. Data list: 130
Finished N16_rv0.5_rg8.0_z0.02. All data: 7044. Data list: 612
Finished N16_rv1.0_rg2.0_z0.0002. All data: 7065. Data list: 21
Finished N16_rv1.0_rg2.0_z0.002. All data: 7099. Data list: 34
Finished N16_rv1.0_rg2.0_z0.02. All data: 7235. Data list: 136
Finished N16_rv1.0_rg20.0_z0.0002. All data: 7254. Data list: 19
Finished N16_rv1.0_rg20.0_z0.002. All data: 7295. Data list: 41
Finished N16_rv1.0_rg20.0_z0.02. All data: 7418. Data list: 123
Finished N16_rv1.0_rg8.0_z0.0002. All data: 7439. Data list: 21
Finished N16_rv1.0_rg8.0_z0.002. All data: 7480. Data list: 41
Finished N16_rv1.0_rg8.0_z0.02. All data: 7612. Data list: 132
Finished N16_rv2.0_rg2.0_z0.0002. All data: 

In [167]:
headers = ["N", "rv", "rg", "z", "t_snapshot[myr]", "id", "m[MSUN]", "r", "vr", "vt", "E", "J", "binflag", "m0[MSUN]", "m1[MSUN]", "id0", "id1", "a[AU]", "e", "startype", "luminosity[LSUN]", "radius[RSUN]", "bin_startype0", "bin_startype1", "bin_star_lum0[LSUN]", "bin_star_lum1[LSUN]", "bin_star_radius0[RSUN]", "bin_star_radius1[RSUN]", "bin.Eb", "eta", "star.phi", "rad0", "rad1", "tb", "lum0", "lum1", "massc0", "massc1", "radc0", "radc1", "menv0", "menv1", "renv0", "renv1", "tms0", "tms1", "dmdt0", "dmdt1", "radrol0", "radrol1", "ospin0", "ospin1", "B0", "B1", "formation0", "formation1", "bacc0", "bacc1", "tacc0", "tacc1", "mass0_0", "mass0_1", "epoch0", "epoch1", "ospin", "B", "formation", "tcoll"]


with open(output_file_initial_all, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    writer.writerows(all_data_all)

In [168]:
# Load the CSV file into a numpy array
initial_nocollision_data = np.loadtxt(output_file_initial, delimiter=',', dtype=str, skiprows=1)

# Load the CSV file into a numpy array
nocollision_data = np.loadtxt(output_file_nocollision, delimiter=',', dtype=str, skiprows=1)



initial_nocollision_ids = initial_nocollision_data[::, 5]

nocollision_ids = nocollision_data[::, 10]



missing_ids = (set(initial_nocollision_ids) ^ set(nocollision_ids))

print(len(missing_ids))

7


In [169]:
# Load the CSV file into a numpy array
initial_nocollision_data = np.loadtxt(output_file_initial, delimiter=',', dtype=str, skiprows=1)

# Load the CSV file into a numpy array
nocollision_data = np.loadtxt(output_file_nocollision, delimiter=',', dtype=str, skiprows=1)

initial_ids = [x for x in np.concatenate((initial_nocollision_data[::, 5], initial_nocollision_data[::, 15], initial_nocollision_data[::, 16])) if float(x) != -100]

late_ids = nocollision_data[::, 10]

print(len(set(initial_ids)))
print(len(set(late_ids)))

4621
4620


In [170]:
final_list = [i for i in initial_ids if i in late_ids]

In [171]:
print(len(set(final_list)))

### this shows that the number of ids are the same in the initial and late files. the difference in length is attributed to binaries. 

4617


In [172]:
import collections
print([item for item, count in collections.Counter(late_ids).items() if count > 1])

### this actually shows duplicates, not missing values

['90640', '187389', '557173', '394180', '261928', '810261', '750418', '714651', '191780', '615852', '445911', '402272', '137504', '210951', '369869', '809511', '275238', '562093', '1212753', '342876', '24277', '818795', '162611', '231448', '398995', '135356', '182941', '478812', '762451', '96658', '1526498', '1469597', '210253', '487663', '211742', '450335', '727053', '301153', '266957', '192818', '620208', '358229', '172010', '294177', '13921', '362191', '85937', '688215', '180617', '124479', '509124', '10893', '686152', '194172', '461911', '129198', '703783', '154985', '103496', '204608', '340413', '128097', '56897', '158990', '125972', '371212', '58532', '663276', '943042', '41737', '165645', '716777', '196747', '568618', '372036', '277670', '204762', '90388', '571750', '103696', '94651', '309577', '170577', '152672', '357448', '209416', '611066', '1584613', '56583', '694915', '278101', '731876', '205448', '67040', '5627', '89406', '154983', '163908', '102231', '356027', '172062', '

In [173]:
data_path = 'data/updated_simulation_data'
output_file_all_all = 'processed_data/collisions_all_data.csv'
output_file_all = 'processed_data/allcollisions_GiantBH_data.csv'
output_file_nocollision = 'processed_data/nocollision_GiantBH_data.csv'
output_file_iscollision = 'processed_data/iscollision_GiantBH_data.csv'
#header = "#N,#rv,#rg,#z,#t_snapshot[myr],#M1[MSUN],#M2[MSUN],#k1,#k2,#id1,#id2,#sma[AU],#ecc,#bin_star_radius0[RSUN],#bin_star_radius1[RSUN],#snapshot, #roche_lobe1_calc[RSUN], #roche_lobe2_calc[RSUN],#radrol0,#radrol1"

output_file_initial = 'processed_data/initial_GiantBH_data.csv'
output_file_initial_all = 'processed_data/initial_all_GiantBH_data.csv'


In [174]:
initial_data = np.loadtxt(output_file_initial, delimiter=',', dtype=str, skiprows=1)
